In [33]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import numpy as np

In [34]:
data_path_csv = "data/mq_variants_intensity.csv"
mq_variants_df = pd.read_csv(data_path_csv, low_memory=False)

In [35]:
mq_variants_df.head()

,rowid,ccms_row_id,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,...,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod
0,1,1,.IITHPNFNGNTLDNDIM+15.995LIK.,37658,.IITHPNFNGNTLDNDIMLIK.,11683,20735,81,TRYP_PIG,2299.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,.VADPDHDHTGFLTEYVATR.,93378,.VADPDHDHTGFLTEYVATR.,11372,15019,62,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,2144.0,...,"182,810,000","182,810,000","296,340,000","296,340,000","272,890,000","272,890,000","254,860,000","254,860,000","70,792,000","70,792,000"
2,3,3,.LGEHNIDVLEGNEQFINAAK.,50733,.LGEHNIDVLEGNEQFINAAK.,8878,23098,134,TRYP_PIG,2211.1,...,"152,910,000","152,910,000","313,690,000","313,690,000","187,600,000","187,600,000","313,290,000","313,290,000","204,790,000","204,790,000"
3,4,4,.FRHENIIGINDIIR.,25741,.FRHENIIGINDIIR.,8720,12619,33,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,1709.9,...,"115,160,000","115,160,000","223,460,000","223,460,000","182,890,000","182,890,000","236,530,000","236,530,000","97,725,000","97,725,000"
4,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,"19,220,000","19,220,000","11,216,000","11,216,000","12,721,000","12,721,000","12,835,000","12,835,000","8,137,600","8,137,600"


In [36]:
# Define a function to convert columns with comma-separated numbers to numeric
def convert_comma_separated_to_numeric(df: pd.DataFrame):
    """
    Convert columns with comma-separated numbers to numeric type.
    """
    # Find columns that might contain string representations of numbers
    string_cols = df.select_dtypes(include=["object"]).columns

    # Track conversion statistics
    converted = 0
    failed = 0

    for col in string_cols:
        # Skip columns that are clearly not numeric intensity values
        if not col.startswith("_dyn_"):
            continue

        try:
            # First, try direct conversion (handles NaN and already numeric values)
            temp_series = df[col].str.replace(",", "", regex=False)
            # Convert to numeric, forcing errors to NaN
            temp_series = pd.to_numeric(temp_series, errors="raise")

            # If there are non-null values, replace the column
            if not temp_series.isna().all():
                df[col] = temp_series
                converted += 1

        except Exception as e:
            print(f"Could not convert column: {col}, Error: {str(e)}")
            failed += 1

    print(
        f"Converted {converted} columns to numeric. Failed to convert {failed} columns."
    )
    return df


# Process the dataset
mq_variants_df = convert_comma_separated_to_numeric(mq_variants_df)

# Display info to verify conversion
print("\nDataset summary after conversion:")
print(mq_variants_df.dtypes.value_counts())

Converted 1000 columns to numeric. Failed to convert 0 columns.

Dataset summary after conversion:
float64    1008
object       14
int64        10
bool          1
Name: count, dtype: int64


In [37]:
# Identify all treatment columns (those starting with "_dyn_")
treatment_cols = [col for col in mq_variants_df.columns if col.startswith('_dyn_')]

# Check which rows have all NaN values in the treatment columns
rows_to_keep = ~mq_variants_df[treatment_cols].isna().all(axis=1)

# Filter the dataframe to keep only rows with at least one treatment value
filtered_mq_variants_df = mq_variants_df[rows_to_keep].reset_index(drop=True)

# Print information about the filtering
print(f"Original number of rows: {mq_variants_df.shape[0]}")
print(f"Rows after filtering: {filtered_mq_variants_df.shape[0]}")
print(f"Removed {mq_variants_df.shape[0] - filtered_mq_variants_df.shape[0]} rows ({(mq_variants_df.shape[0] - filtered_mq_variants_df.shape[0]) / mq_variants_df.shape[0] * 100:.2f}% of total)")

# Replace the original dataframe with the filtered one
mq_variants_df = filtered_mq_variants_df

Original number of rows: 83706
Rows after filtering: 50659
Removed 33047 rows (39.48% of total)


In [38]:
mq_variants_df.drop(columns=['rowid', 'ccms_row_id'], inplace=True, errors='ignore')
mq_variants_df.head()

,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,Charge,Num Mods,...,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod
0,.VADPDHDHTGFLTEYVATR.,93378,.VADPDHDHTGFLTEYVATR.,11372,15019,62,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,2144.0,2,0,...,182810000.0,182810000.0,296340000.0,296340000.0,272890000.0,272890000.0,254860000.0,254860000.0,70792000.0,70792000.0
1,.LGEHNIDVLEGNEQFINAAK.,50733,.LGEHNIDVLEGNEQFINAAK.,8878,23098,134,TRYP_PIG,2211.1,2,0,...,152910000.0,152910000.0,313690000.0,313690000.0,187600000.0,187600000.0,313290000.0,313290000.0,204790000.0,204790000.0
2,.FRHENIIGINDIIR.,25741,.FRHENIIGINDIIR.,8720,12619,33,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,1709.9,2,0,...,115160000.0,115160000.0,223460000.0,223460000.0,182890000.0,182890000.0,236530000.0,236530000.0,97725000.0,97725000.0
3,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,2,0,...,19220000.0,19220000.0,11216000.0,11216000.0,12721000.0,12721000.0,12835000.0,12835000.0,8137600.0,8137600.0
4,.NYLLSLPHK.,68115,.NYLLSLPHK.,7445,11842,41,sp|P28482|MK01_HUMAN,1084.6,2,0,...,290970000.0,290970000.0,477300000.0,477300000.0,363140000.0,363140000.0,43697000.0,43697000.0,182850000.0,182850000.0


In [39]:
mq_variants_df.drop(columns=[col for col in mq_variants_df.columns if col.startswith('_dyn_') and col.endswith('_unmod')], inplace=True)

In [40]:
# To CSV
mq_variants_df.to_csv("data/mq_variants_intensity_cleaned.csv", index=False)